In [1]:
import pandas as pd
import numpy as np
from time import time

from IPython.display import display

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def canopy(X, T1, T2, distance_metric='euclidean', filemap=None):
    canopies = dict()
    X1_dist = pairwise_distances(X, metric=distance_metric)
    canopy_points = set(range(X.shape[0]))
    while canopy_points:
        point = canopy_points.pop()
        i = len(canopies)
        canopies[i] = {"c":point, "points": list(np.where(X1_dist[point] < T2)[0])}
        canopy_points = canopy_points.difference(set(np.where(X1_dist[point] < T1)[0]))
    if filemap:
        for canopy_id in canopies.keys():
            canopy = canopies.pop(canopy_id)
            canopy2 = {"c":filemap[canopy['c']], "points":list()}
            for point in canopy['points']:
                canopy2["points"].append(filemap[point])
            canopies[canopy_id] = canopy2
    return canopies

In [3]:
def euclideanDistance(vector1, vector2):
        #print(vector1)
        #print(vector2)
        return np.sqrt(np.sum(np.power(vector1-vector2, 2)))

def getDistance(row_center, row_sample):
        #print(row_center)
        row_center = np.asarray(row_center)
        #row_center = np.asarray(row_sample)
        return euclideanDistance(row_center, row_sample)

def getSquaredError(data, kmeans):
    distances = []
    for i in range(k): # Qtd de clusters
        distance = 0
        for index_labels, value_labels in enumerate(kmeans.labels_): #kmeans.labels_ possui o cluster de cada elemento
            if (i == value_labels):
                #print(value_labels)
                distance = distance + getDistance(kmeans.cluster_centers_[value_labels], data.loc[index_labels].values)
        distances.append(distance) #Erro quadratico medio de cada cluster
    distances = np.asarray(distances)
    error = np.sum(distances)
    return error

In [4]:
# -------------- Density Canopy -------------- #

# Definition 1
#OBS.: enumerate com numpy mto mais rápido que iterrows
def mean_dist(D):
    n = D.shape[0]
    D = D.values
    sum_D = np.zeros((n, n))
    for i, row_i in enumerate(D):
        for j, row_j in enumerate(D[i+1:,]):
            sum_D[i][j] = euclideanDistance(row_i, row_j)
    return (2/(n*(n-1))) * np.sum(sum_D)

# Definition 2
def get_densities(D, meanDis):
    densities = np.zeros(D.shape[0], dtype=int)
    aux_D = D.values
    for i, row_i in enumerate(aux_D):
        for j, row_j in enumerate(aux_D):
            if euclideanDistance(row_i, row_j) - meanDis < 0:
                densities[i] += 1
    df = pd.DataFrame(data=densities, columns=["density"])
    return df

#Definition 3
def cluster_dist_mean(D, densities, meanDis):
    a = np.zeros(D.shape[0])
    densities_aux = densities.copy().values
    for i, row_i in enumerate(D.values):
        sum_dists = 0
        for j, row_j in enumerate(D.values):
            dist = euclideanDistance(row_i, row_j)
            if dist - meanDis < 0:
                sum_dists += dist
        a[i] = (2/(densities_aux[i]*(densities_aux[i]-1))) * sum_dists
    return a

#Definition 4
def clusters_dist(D, densities):
    s = []
    densities_aux = densities.values
    for i, row_i in enumerate(D.values):
        maxDist = 0
        minDist = float("inf")
        dist = 0
        flag = 1 #Se flag=0 entao min dist, se flag=1 retornar max dist
        for j, row_j in enumerate(D.values):
            dist = euclideanDistance(row_i, row_j)
            if densities_aux[j] > densities_aux[i]:
                flag = 0
                if (dist < minDist):
                    minDist = dist
            else:
                if (dist > maxDist):
                    maxDist = dist
#             if densities[j] == np.amax(densities):
#                 flag = 1
        if flag == 1:
            s.append(maxDist)
        else: # p(j) > p(i)
            s.append(minDist)
    return s

# Definition 6
def product_weight(p, a, s):
    w = []
    for i, row_i in enumerate(p):
        w.append(p[i] * (1/a[i]) * s[i])
    return w
        
def getCluster(D, meanDis, firstExecution=True, index=None): 
    aux_D = D.copy()
    df = get_densities(D, meanDis)
    if (firstExecution == True): #Primeira execução pega o de maior densidade
        rows = D.iloc[df.idxmax(axis=0)].values
        row_i = rows[-1] #pegando o último index com maior peso, caso tenha mais de 1
    else:
        row_i = D.iloc[index].values #pegando index passado, no caso o com maior peso
    cluster = []
    for j, row_j in enumerate(aux_D.values):
        if euclideanDistance(row_i, row_j) - meanDis < 0:
            cluster.append(j)
    #print (cluster)
    #auxD.drop(cluster, inplace=True)
    #display(auxD)
    return row_i, cluster #Elemento central e cluster

def removeOutliers(aux_D, densities, s, meanDis):
    #remove elemento com densidade = 1 e que o s[i] seja maior que o raio
    outliers = []
    for i, row_i in enumerate(aux_D.values):
        if densities[i] == 1 and s[i] > meanDis:
            outliers.append(i)
    aux_D.drop(outliers, inplace=True) #removendo outliers
    aux_D.reset_index(drop=True, inplace=True)
    densities = np.delete(densities, outliers, 0)
    s = np.delete(s, outliers, 0)
    return aux_D, densities, s

def run(D):
    meanDis = mean_dist(D)
    centers, cluster = getCluster(D, meanDis)
    centers = np.array([centers])
    aux_D = D.copy()
    aux_D.drop(cluster, inplace=True) #removendo cluster ja identificado
    aux_D.reset_index(drop=True, inplace=True)
    while (not aux_D.empty):
        new_densities = get_densities(aux_D, meanDis)
        a = cluster_dist_mean(aux_D, new_densities, meanDis) #mantem-se meanDis
        s = clusters_dist(aux_D, new_densities)
        densities = new_densities.values
        aux_D, densities, s = removeOutliers(aux_D, densities, s, meanDis)
        aux_index = np.argmax(product_weight(densities, a, s))
#         for i, row_i in enumerate(aux_D.values):
#             if (densities[i] * (1/a[i]) * s[i]) > weight:
#                 weight = densities[i] * (1/a[i]) * s[i]
#                 print("densidade: ", densities[i], "a: ", a[i], "s: ", s[i], "weight: ", weight)
#                 aux_index = i
        aux_center, cluster = getCluster(aux_D, meanDis, firstExecution=False, index=aux_index)
        aux_center = np.array([aux_center])
        centers = np.concatenate((centers, aux_center), axis=0)
        aux_D.drop(cluster, inplace=True) #removendo cluster ja identificado
        aux_D.reset_index(drop=True, inplace=True)
    print("KS:", len(centers), centers)


In [5]:
files = ["soybean-small", "iris", "wine",  "segmentation", "ionosphere"]
ks = [4,3,3,7,2]
#kmeansTypes = ["random", "k-means++"]
kmeansTypes = ["random"]

In [6]:
for kmeansType in kmeansTypes:
    print ("--------- "+ kmeansType +" test ---------")
    for index, file in enumerate(files):
        print ("\n----- "+file+" -----\n")
        data = pd.read_csv("datasets/"+file+".data", header=None)
        print (data.shape)
        if file == "segmentation": #Target eh na primeira coluna
            target = data.iloc[:,0]
            data = data.iloc[:,1:]   
        else: #Target na última coluna
            target = data.iloc[:,-1]
            data = data.iloc[:,:-1]
        run(data)
        clustering_times = []
        start = time()
        k = ks[index]
        kmeans = KMeans(n_clusters=k, random_state=100, init=kmeansType, n_init=1, max_iter=100).fit(data)
        #display(data)
        error = getSquaredError(data, kmeans)
        #print(kmeans.labels_)
        end = time()
        T1 = error/(data.shape[0])
        T2 = error/(data.shape[0]/2)
        
        #print(canopy(data.values, T1, T2))
        #print("Erro quadrático médio: ",error)
        clustering_times.append(end - start)
        #print(clustering_times)
        #print(kmeans.cluster_centers_)

--------- random test ---------

----- soybean-small -----

(47, 36)
KS: 3 [[3 1 2 0 0 2 1 2 1 1 1 1 0 2 2 0 0 0 1 0 2 2 0 0 0 0 0 3 4 0 0 0 0 0 1]
 [4 0 0 1 0 2 3 1 1 1 1 1 0 2 2 0 0 0 1 0 0 3 0 0 0 2 1 0 4 0 0 0 0 0 0]
 [5 0 2 1 0 3 1 1 1 2 1 1 0 2 2 0 0 0 1 1 3 0 1 1 0 0 0 0 4 0 0 0 0 0 0]]

----- iris -----

(150, 5)
KS: 3 [[5.6 2.9 3.6 1.3]
 [4.7 3.2 1.3 0.2]
 [6.9 3.2 5.7 2.3]]

----- wine -----

(178, 14)
KS: 4 [[2.000e+00 1.196e+01 1.090e+00 2.300e+00 2.100e+01 1.010e+02 3.380e+00
  2.140e+00 1.300e-01 1.650e+00 3.210e+00 9.900e-01 3.130e+00]
 [1.000e+00 1.376e+01 1.530e+00 2.700e+00 1.950e+01 1.320e+02 2.950e+00
  2.740e+00 5.000e-01 1.350e+00 5.400e+00 1.250e+00 3.000e+00]
 [2.000e+00 1.225e+01 1.730e+00 2.120e+00 1.900e+01 8.000e+01 1.650e+00
  2.030e+00 3.700e-01 1.630e+00 3.400e+00 1.000e+00 3.170e+00]
 [2.000e+00 1.221e+01 1.190e+00 1.750e+00 1.680e+01 1.510e+02 1.850e+00
  1.280e+00 1.400e-01 2.500e+00 2.850e+00 1.280e+00 3.070e+00]]

----- segmentation -----

(210, 20)


/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in multiply


KS: 4 [[ 1.4000000e+02  7.3000000e+01  9.0000000e+00  0.0000000e+00
   0.0000000e+00  1.7222214e+00  8.2775980e-01  7.7777740e-01
   1.0036957e+00  4.6296295e+01  4.5666668e+01  5.1111110e+01
   4.2111110e+01 -1.8888888e+00  1.4444445e+01 -1.2555555e+01
   5.1111110e+01  1.7615560e-01 -1.6815889e+00]
 [ 4.0000000e+00  1.8900000e+02  9.0000000e+00  0.0000000e+00
   0.0000000e+00  2.0555565e+00  3.8851852e+00  1.1722221e+01
   1.1459634e+02  2.6444445e+01  2.3444445e+01  3.3000000e+01
   2.2888890e+01 -9.0000000e+00  1.9666666e+01 -1.0666667e+01
   3.3000000e+01  2.7147257e-01 -2.1010017e+00]
 [ 1.2400000e+02  2.9000000e+01  9.0000000e+00  0.0000000e+00
   0.0000000e+00  1.0000013e+00  7.1110760e-01  1.0555534e+00
   9.0741664e-01  1.2844444e+02  1.1922222e+02  1.4288889e+02
   1.2322222e+02 -2.7666666e+01  4.3333332e+01 -1.5666667e+01
   1.4288889e+02  1.6561614e-01 -2.2711280e+00]
 [ 1.9700000e+02  2.3600000e+02  9.0000000e+00  0.0000000e+00
   0.0000000e+00  2.4444444e+00  6.8296280e+

/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide
/home/pgdock/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in multiply


KS: 7 [[ 1.       0.       0.21429 -0.09524  0.33333  0.07143  0.19048  0.19048
   0.2381   0.09524  0.40476  0.02381  0.30952 -0.04762  0.30952 -0.04762
   0.28571 -0.11905  0.33333  0.04762  0.30952  0.       0.21429 -0.11905
   0.35714 -0.04762  0.22109 -0.0229   0.19048  0.       0.16997 -0.02034
   0.14694 -0.01877]
 [ 1.       0.       1.      -0.5421   1.      -1.       1.      -1.
   1.       0.36217  1.      -0.41119  1.       1.       1.      -1.
   1.      -0.29354  1.      -0.93599  1.       1.       1.       1.
   1.      -0.40888  1.      -0.62745  1.      -1.       1.      -1.
   1.      -1.     ]
 [ 1.       0.       0.9287   0.33164  0.76168  0.62349  0.49305  0.84266
   0.21592  0.95193 -0.13956  0.96167 -0.47202  0.8359  -0.70747  0.6549
  -0.87474  0.3675  -0.91814  0.05595 -0.89824 -0.26173 -0.73969 -0.54069
  -0.50757 -0.74735 -0.22323 -0.86122  0.0781  -0.87159  0.36021 -0.78057
   0.59407 -0.6027 ]
 [ 1.       0.       1.       0.06843  1.       0.14211  1.     